# Sentiment Analysis

### Author Sara Mezuri

In [2]:
# Importing the Packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set()

# Bloomberg Terminal Data

Fetching sentiment data for news was one of the biggest challenges in this project. Many news APIs had strict limitations, required subscriptions, or were costly. I also tried web scraping, but it only allowed me to gather data for up to one month. Finally, I turned to the Bloomberg Terminal.

The Bloomberg Terminal provided both news and Twitter sentiment data starting from January 2015. It included daily counts of positive and negative sentiment, which I used to calculate the sentiment scores for the two indices selected in this project.

Another challenge was that Bloomberg didn’t provide sentiment data directly for the indices. Instead, I had to manually retrieve the data for individual stocks. Since one index contains 100 stocks and the other has 500, manually collecting each dataset would have been extremely time-consuming. To simplify this, I decided to calculate the sentiment scores for each index as a weighted average of the 15 most influential stocks, with their influence determined by market capitalization.

## Nvidia News Sentiment Data Bloomberg Terminal

![Nvidia Stock Bloomberg Terminal](NVDA2.jpg)

![Nvidia Stock Bloomberg Terminal](NVDA.jpg)

## Nvidia News Sentiment Data as seen in Bloomberg Terminal
![Sentiment Data Bloomberg Terminal](sentiment_bloomberg.gif)

As you can see from the image above, Bloomberg Terminal provides **absolute counts** of news articles or tweets tagged as positive or negative. 

The Bloomberg Terminal processes news and social media data using **natural language processing (NLP)** algorithms. Each article or tweet is scanned for sentiment-related keywords, tone, and context, and classified as positive, negative, or neutral.

* Positive Count: The number of articles/tweets with positive sentiment.

* Negative Count: The number of articles/tweets with negative sentiment.

The numbers shown here (e.g., 6 for positive and -12 for negative) are **not sentiment scores** in the weighted or normalized sense. They are **raw counts** of articles or tweets identified as positive or negative.

## Apple News Sentiment Data Bloomberg Terminal
![Apple News Data Bloomberg Terminal](AAPL_Aug_2024.jpg)

## Tesla Twitter Sentiment Data Bloomberg Terminal
![Tesla Twitter Sentiment Data Bloomberg Terminal](TSLA.jpg)

# Most influential stocks

The next step was to determine the top 15 most influential companies for each index. I used [FinanceCharts S&P 500 Companies](https://www.financecharts.com/screener/sp-500) and [FinanceCharts Nasdaq 100 Companies](https://www.financecharts.com/screener/nasdaq-100)
 to get the market capitalization data and calculated each company’s percentage share.



In [ ]:
# Import the S&P 500 Companies data
sp500_companies = pd.read_excel("../Stock_Data/S&P500Companies.xlsx")

# Check the first 10 stocks
sp500_companies.head(n=15)

,Name,Ticker,Sector,Market Cap,Close Price,Price Change,TTM Revenue,TTM Net Income,TTM Total Rtn,Div Yield,PE Ratio
0,Apple Inc,AAPL,Information Technology,3.624087e+12,237.33,1.022,391035000000,93736000000,25.939,0.421,39.035
1,NVIDIA Corp,NVDA,Information Technology,3.458541e+12,138.25,2.150,113269000000,63074000000,187.267,0.029,54.429
2,Microsoft Corp,MSFT,Information Technology,3.151873e+12,423.46,0.111,254190000000,90512000000,12.611,0.784,34.939
3,Amazon.com Inc,AMZN,Consumer Discretionary,2.208809e+12,207.89,1.045,620128000000,49868000000,42.079,0.000,44.516
4,Alphabet Inc,GOOGL,Communication Services,2.076902e+12,168.95,-0.165,339859000000,94269000000,25.468,0.355,22.407
5,Meta Platforms Inc,META,Communication Services,1.449584e+12,574.32,0.900,156226000000,55539000000,73.404,0.261,27.091
6,Tesla Inc,TSLA,Consumer Discretionary,1.148823e+12,345.16,3.686,97150000000,12743000000,41.378,0.000,94.564
7,Berkshire Hathaway Inc,BRK.B,Financials,1.041835e+12,483.02,-0.012,369893000000,106875000000,34.662,0.000,9.758
8,Eli Lilly and Co,LLY,Health Care,7.618971e+11,795.35,0.908,40863300000,8369900000,35.259,0.654,85.984
9,Broadcom Inc,AVGO,Information Technology,7.570070e+11,162.08,1.509,46815000000,5095000000,74.928,1.308,140.939


In [44]:
# Calculate the total market capitalization for all companies
total_market_cap_sp = sp500_companies['Market Cap'].sum()

# Calculate the percentage of market cap for each stock
sp500_companies['Market Cap Percentage'] = (sp500_companies['Market Cap'] / total_market_cap_sp) * 100

# Check the results
sp500_companies.head(n=15)

,Name,Ticker,Sector,Market Cap,Close Price,Price Change,TTM Revenue,TTM Net Income,TTM Total Rtn,Div Yield,PE Ratio,Market Cap Percentage
0,Apple Inc,AAPL,Information Technology,3.624087e+12,237.33,1.022,391035000000,93736000000,25.939,0.421,39.035,6.653350
1,NVIDIA Corp,NVDA,Information Technology,3.458541e+12,138.25,2.150,113269000000,63074000000,187.267,0.029,54.429,6.349429
2,Microsoft Corp,MSFT,Information Technology,3.151873e+12,423.46,0.111,254190000000,90512000000,12.611,0.784,34.939,5.786427
3,Amazon.com Inc,AMZN,Consumer Discretionary,2.208809e+12,207.89,1.045,620128000000,49868000000,42.079,0.000,44.516,4.055085
4,Alphabet Inc,GOOGL,Communication Services,2.076902e+12,168.95,-0.165,339859000000,94269000000,25.468,0.355,22.407,3.812920
5,Meta Platforms Inc,META,Communication Services,1.449584e+12,574.32,0.900,156226000000,55539000000,73.404,0.261,27.091,2.661247
6,Tesla Inc,TSLA,Consumer Discretionary,1.148823e+12,345.16,3.686,97150000000,12743000000,41.378,0.000,94.564,2.109090
7,Berkshire Hathaway Inc,BRK.B,Financials,1.041835e+12,483.02,-0.012,369893000000,106875000000,34.662,0.000,9.758,1.912673
8,Eli Lilly and Co,LLY,Health Care,7.618971e+11,795.35,0.908,40863300000,8369900000,35.259,0.654,85.984,1.398744
9,Broadcom Inc,AVGO,Information Technology,7.570070e+11,162.08,1.509,46815000000,5095000000,74.928,1.308,140.939,1.389766


In [ ]:
# Import the Nasdaq 100 Companies data
nasdaq100_companies = pd.read_excel("../Stock_Data/Nasdaq100Companies.xlsx")

# Check the first 10 stocks
nasdaq100_companies.head(n=15)

,Name,Ticker,Sector,Market Cap,Close Price,Price Change,TTM Revenue,TTM Net Income,TTM Total Rtn,Div Yield,PE Ratio
0,Apple Inc,AAPL,Information Technology,3.624087e+12,237.33,1.022,3.910350e+11,9.373600e+10,25.939,0.421,39.035
1,NVIDIA Corp,NVDA,Information Technology,3.458541e+12,138.25,2.150,1.132690e+11,6.307400e+10,187.267,0.029,54.429
2,Microsoft Corp,MSFT,Information Technology,3.151873e+12,423.46,0.111,2.541900e+11,9.051200e+10,12.611,0.784,34.939
3,Amazon.com Inc,AMZN,Consumer Discretionary,2.208809e+12,207.89,1.045,6.201280e+11,4.986800e+10,42.079,0.000,44.516
4,Alphabet Inc,GOOGL,Communication Services,2.076902e+12,168.95,-0.165,3.398590e+11,9.426900e+10,25.468,0.355,22.407
5,Meta Platforms Inc,META,Communication Services,1.449584e+12,574.32,0.900,1.562260e+11,5.553900e+10,73.404,0.261,27.091
6,Tesla Inc,TSLA,Consumer Discretionary,1.148823e+12,345.16,3.686,9.715000e+10,1.274300e+10,41.378,0.000,94.564
7,Broadcom Inc,AVGO,Information Technology,7.570070e+11,162.08,1.509,4.681500e+10,5.095000e+09,74.928,1.308,140.939
8,Costco Wholesale Corp,COST,Consumer Staples,4.352404e+11,971.88,1.074,2.544530e+11,7.367000e+09,70.072,0.477,58.653
9,Netflix Inc,NFLX,Communication Services,3.790742e+11,886.81,1.079,3.758728e+10,7.780862e+09,85.840,0.000,50.187


In [9]:
# Calculate the total market capitalization for all companies
total_market_cap_ndx = nasdaq100_companies['Market Cap'].sum()

# Calculate the percentage of market cap for each stock
nasdaq100_companies['Market Cap Percentage'] = (nasdaq100_companies['Market Cap'] / total_market_cap_ndx) * 100

# Check the results
nasdaq100_companies.head(n=15)

,Name,Ticker,Sector,Market Cap,Close Price,Price Change,TTM Revenue,TTM Net Income,TTM Total Rtn,Div Yield,PE Ratio,Market Cap Percentage
0,Apple Inc,AAPL,Information Technology,3.624087e+12,237.33,1.022,3.910350e+11,9.373600e+10,25.939,0.421,39.035,13.624913
1,NVIDIA Corp,NVDA,Information Technology,3.458541e+12,138.25,2.150,1.132690e+11,6.307400e+10,187.267,0.029,54.429,13.002535
2,Microsoft Corp,MSFT,Information Technology,3.151873e+12,423.46,0.111,2.541900e+11,9.051200e+10,12.611,0.784,34.939,11.849604
3,Amazon.com Inc,AMZN,Consumer Discretionary,2.208809e+12,207.89,1.045,6.201280e+11,4.986800e+10,42.079,0.000,44.516,8.304115
4,Alphabet Inc,GOOGL,Communication Services,2.076902e+12,168.95,-0.165,3.398590e+11,9.426900e+10,25.468,0.355,22.407,7.808202
5,Meta Platforms Inc,META,Communication Services,1.449584e+12,574.32,0.900,1.562260e+11,5.553900e+10,73.404,0.261,27.091,5.449775
6,Tesla Inc,TSLA,Consumer Discretionary,1.148823e+12,345.16,3.686,9.715000e+10,1.274300e+10,41.378,0.000,94.564,4.319052
7,Broadcom Inc,AVGO,Information Technology,7.570070e+11,162.08,1.509,4.681500e+10,5.095000e+09,74.928,1.308,140.939,2.846001
8,Costco Wholesale Corp,COST,Consumer Staples,4.352404e+11,971.88,1.074,2.544530e+11,7.367000e+09,70.072,0.477,58.653,1.636305
9,Netflix Inc,NFLX,Communication Services,3.790742e+11,886.81,1.079,3.758728e+10,7.780862e+09,85.840,0.000,50.187,1.425146


## Loading all the sentiment data for each stock

Based on the data above, I selected the stocks and collected the news and Twitter sentiment data for each. Then, I created a main dataset that combines all the sentiment data.

To calculate the sentiment score for your dataset, we can assume that the sentiment score is computed using the positive and negative sentiment counts for both news and Twitter data.

The formula for calculating the sentiment score could be:

**Sentiment Score = (Positive Sentiment Count − Negative Sentiment Count) / (Total Publications)**

​where:

*Positive Sentiment Count* is the count of positive sentiment posts.

*Negative Sentiment Count* is the count of negative sentiment posts.

**NOTE**:  In the Bloomberg Terminal, the count of negative sentiment posts is represented as a negative integer, even though counts are inherently positive values. To account for this, when calculating the sentiment score, I will *add* the Positive Sentiment Count and Negative Sentiment Count instead of subtracting them. This avoids converting negative counts into positive values, ensuring the sentiment score accurately reflects the overall sentiment direction.

In [ ]:
# Import the Apple data
apple = pd.read_excel("../Sentiment_Data/AAPL.xlsx")

# Ensure 'Date' is in datetime format
apple['Date'] = pd.to_datetime(apple['Date'], errors='coerce')

# Set 'Date' as the index
apple.set_index('Date', inplace=True)

# Calculate sentiment score for News
apple['News Sentiment Score'] = (apple['News Positive Sentiment Count'] + apple['News Negative Sentiment Count']) / apple['News Publication Count']

# Calculate sentiment score for Twitter
apple['Twitter Sentiment Score'] = (apple['Twitter Positive Sentiment Count'] + apple['Twitter Negative Sentiment Count']) / apple['Twitter Publication Count']

# Display the dataframe with the sentiment scores
apple.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,501.0,3.542564e+12,3.542564e+12,3.542564e+12,3.542564e+12,1.0,-4.0,185.0,3.542564e+12,3.542564e+12,3.542564e+12,3.542564e+12,3.0,-11.0,-0.005988,-0.043243
2024-09-27,642.0,3.463350e+12,3.463350e+12,3.463350e+12,3.463350e+12,3.0,-5.0,303.0,3.463350e+12,3.463350e+12,3.463350e+12,3.463350e+12,4.0,-16.0,-0.003115,-0.039604
2024-09-26,798.0,3.459245e+12,3.459245e+12,3.459245e+12,3.459245e+12,4.0,-24.0,368.0,3.459245e+12,3.459245e+12,3.459245e+12,3.459245e+12,9.0,-28.0,-0.025063,-0.051630
2024-09-25,817.0,3.441760e+12,3.441760e+12,3.441760e+12,3.441760e+12,13.0,-7.0,304.0,3.441760e+12,3.441760e+12,3.441760e+12,3.441760e+12,7.0,-21.0,0.007344,-0.046053
2024-09-24,981.0,3.456965e+12,3.456965e+12,3.456965e+12,3.456965e+12,8.0,-9.0,480.0,3.456965e+12,3.456965e+12,3.456965e+12,3.456965e+12,7.0,-19.0,-0.001019,-0.025000


In [ ]:
# Import the Adobe data
adobe = pd.read_excel("../Sentiment_Data/ADBE.xlsx")

# Ensure 'Date' is in datetime format
adobe['Date'] = pd.to_datetime(adobe['Date'], errors='coerce')

# Set 'Date' as the index
adobe.set_index('Date', inplace=True)

# Calculate sentiment score for News
adobe['News Sentiment Score'] = (adobe['News Positive Sentiment Count'] + adobe['News Negative Sentiment Count']) / adobe['News Publication Count']

# Calculate sentiment score for Twitter
adobe['Twitter Sentiment Score'] = (adobe['Twitter Positive Sentiment Count'] + adobe['Twitter Negative Sentiment Count']) / adobe['Twitter Publication Count']

# Display the dataframe with the sentiment scores
adobe.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,15.0,2.279268e+11,2.279268e+11,2.279268e+11,2.279268e+11,0.0,-2.0,11.0,2.279268e+11,2.279268e+11,2.279268e+11,2.279268e+11,0.0,-2.0,-0.133333,-0.181818
2024-09-27,56.0,2.269143e+11,2.269143e+11,2.269143e+11,2.269143e+11,0.0,-4.0,16.0,2.269143e+11,2.269143e+11,2.269143e+11,2.269143e+11,3.0,-1.0,-0.071429,0.125000
2024-09-26,77.0,2.268703e+11,2.268703e+11,2.268703e+11,2.268703e+11,5.0,-6.0,39.0,2.268703e+11,2.268703e+11,2.268703e+11,2.268703e+11,12.0,-2.0,-0.012987,0.256410
2024-09-25,41.0,2.296369e+11,2.296369e+11,2.296369e+11,2.296369e+11,0.0,-4.0,19.0,2.296369e+11,2.296369e+11,2.296369e+11,2.296369e+11,2.0,0.0,-0.097561,0.105263
2024-09-24,20.0,2.323726e+11,2.323726e+11,2.323726e+11,2.323726e+11,6.0,-1.0,17.0,2.323726e+11,2.323726e+11,2.323726e+11,2.323726e+11,1.0,-1.0,0.250000,0.000000


In [ ]:
# Import the ASML Holding data
asml = pd.read_excel("../Sentiment_Data/ASML.xlsx")

# Ensure 'Date' is in datetime format
asml['Date'] = pd.to_datetime(asml['Date'], errors='coerce')

# Set 'Date' as the index
asml.set_index('Date', inplace=True)

# Calculate sentiment score for News
asml['News Sentiment Score'] = (asml['News Positive Sentiment Count'] + asml['News Negative Sentiment Count']) / asml['News Publication Count']

# Calculate sentiment score for Twitter
asml['Twitter Sentiment Score'] = (asml['Twitter Positive Sentiment Count'] + asml['Twitter Negative Sentiment Count']) / asml['Twitter Publication Count']

# Display the dataframe with the sentiment scores
asml.head()


,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,91.0,2.655605e+11,2.655605e+11,2.655605e+11,2.655605e+11,0.0,-2.0,132.0,2.655605e+11,2.655605e+11,2.655605e+11,2.655605e+11,5.0,-5.0,-0.021978,0.000000
2024-09-27,146.0,2.659975e+11,2.659975e+11,2.659975e+11,2.659975e+11,16.0,0.0,484.0,2.659975e+11,2.659975e+11,2.659975e+11,2.659975e+11,24.0,-4.0,0.109589,0.041322
2024-09-26,101.0,2.710795e+11,2.710795e+11,2.710795e+11,2.710795e+11,11.0,-1.0,470.0,2.710795e+11,2.710795e+11,2.710795e+11,2.710795e+11,29.0,-3.0,0.099010,0.055319
2024-09-25,100.0,2.622264e+11,2.622264e+11,2.622264e+11,2.622264e+11,3.0,-1.0,336.0,2.622264e+11,2.622264e+11,2.622264e+11,2.622264e+11,19.0,-4.0,0.020000,0.044643
2024-09-24,106.0,2.562380e+11,2.562380e+11,2.562380e+11,2.562380e+11,2.0,-1.0,213.0,2.562380e+11,2.562380e+11,2.562380e+11,2.562380e+11,9.0,-6.0,0.009434,0.014085


In [ ]:
# Import the Amazon data
amazon = pd.read_excel("../Sentiment_Data/AMZN.xlsx")

# Ensure 'Date' is in datetime format
amazon['Date'] = pd.to_datetime(amazon['Date'], errors='coerce')

# Set 'Date' as the index
amazon.set_index('Date', inplace=True)

# Calculate sentiment score for News
amazon['News Sentiment Score'] = (amazon['News Positive Sentiment Count'] + amazon['News Negative Sentiment Count']) / amazon['News Publication Count']

# Calculate sentiment score for Twitter
amazon['Twitter Sentiment Score'] = (amazon['Twitter Positive Sentiment Count'] + amazon['Twitter Negative Sentiment Count']) / amazon['Twitter Publication Count']

# Display the dataframe with the sentiment scores
amazon.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,167.0,1.955639e+12,1.955639e+12,1.955639e+12,1.955639e+12,1.0,-3.0,110.0,1.955639e+12,1.955639e+12,1.955639e+12,1.955639e+12,5.0,-8.0,-0.011976,-0.027273
2024-09-27,342.0,1.972852e+12,1.972852e+12,1.972852e+12,1.972852e+12,0.0,-20.0,306.0,1.972852e+12,1.972852e+12,1.972852e+12,1.972852e+12,15.0,-15.0,-0.058480,0.000000
2024-09-26,428.0,2.006333e+12,2.006333e+12,2.006333e+12,2.006333e+12,1.0,-9.0,239.0,2.006333e+12,2.006333e+12,2.006333e+12,2.006333e+12,7.0,-14.0,-0.018692,-0.029289
2024-09-25,318.0,2.020711e+12,2.020711e+12,2.020711e+12,2.020711e+12,7.0,-1.0,245.0,2.020711e+12,2.020711e+12,2.020711e+12,2.020711e+12,9.0,-8.0,0.018868,0.004082
2024-09-24,327.0,2.035720e+12,2.035720e+12,2.035720e+12,2.035720e+12,6.0,-2.0,292.0,2.035720e+12,2.035720e+12,2.035720e+12,2.035720e+12,18.0,-4.0,0.012232,0.047945


In [ ]:
# Import the Broadcom data
broadcom = pd.read_excel("../Sentiment_Data/AVGO.xlsx")

# Ensure 'Date' is in datetime format
broadcom['Date'] = pd.to_datetime(broadcom['Date'], errors='coerce')

# Set 'Date' as the index
broadcom.set_index('Date', inplace=True)

# Calculate sentiment score for News
broadcom['News Sentiment Score'] = (broadcom['News Positive Sentiment Count'] + broadcom['News Negative Sentiment Count']) / broadcom['News Publication Count']

# Calculate sentiment score for Twitter
broadcom['Twitter Sentiment Score'] = (broadcom['Twitter Positive Sentiment Count'] + broadcom['Twitter Negative Sentiment Count']) / broadcom['Twitter Publication Count']

# Display the dataframe with the sentiment scores
broadcom.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,40.0,8.056744e+11,8.056744e+11,8.056744e+11,8.056744e+11,0.0,-2.0,38.0,8.056744e+11,8.056744e+11,8.056744e+11,8.056744e+11,3.0,-2.0,-0.050000,0.026316
2024-09-27,230.0,8.065618e+11,8.065618e+11,8.065618e+11,8.065618e+11,19.0,-1.0,95.0,8.065618e+11,8.065618e+11,8.065618e+11,8.065618e+11,2.0,-1.0,0.078261,0.010526
2024-09-26,91.0,8.317829e+11,8.317829e+11,8.317829e+11,8.317829e+11,7.0,-2.0,117.0,8.317829e+11,8.317829e+11,8.317829e+11,8.317829e+11,6.0,0.0,0.054945,0.051282
2024-09-25,80.0,8.197795e+11,8.197795e+11,8.197795e+11,8.197795e+11,7.0,-3.0,105.0,8.197795e+11,8.197795e+11,8.197795e+11,8.197795e+11,5.0,-2.0,0.050000,0.028571
2024-09-24,103.0,8.166035e+11,8.166035e+11,8.166035e+11,8.166035e+11,11.0,0.0,102.0,8.166035e+11,8.166035e+11,8.166035e+11,8.166035e+11,2.0,-2.0,0.106796,0.000000


In [ ]:
# Import the Berkshire Hathaway data
berkhath = pd.read_excel("../Sentiment_Data/BRK.B.xlsx")

# Ensure 'Date' is in datetime format
berkhath['Date'] = pd.to_datetime(berkhath['Date'], errors='coerce')

# Set 'Date' as the index
berkhath.set_index('Date', inplace=True)

# Calculate sentiment score for News
berkhath['News Sentiment Score'] = (berkhath['News Positive Sentiment Count'] + berkhath['News Negative Sentiment Count']) / berkhath['News Publication Count']

# Calculate sentiment score for Twitter
berkhath['Twitter Sentiment Score'] = (berkhath['Twitter Positive Sentiment Count'] + berkhath['Twitter Negative Sentiment Count']) / berkhath['Twitter Publication Count']

# Display the dataframe with the sentiment scores
berkhath.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,11.0,9.923174e+11,9.923174e+11,9.923174e+11,9.923174e+11,0.0,0.0,27.0,9.923174e+11,9.923174e+11,9.923174e+11,9.923174e+11,1.0,-1.0,0.000000,0.000000
2024-09-27,31.0,9.858552e+11,9.858552e+11,9.858552e+11,9.858552e+11,2.0,0.0,26.0,9.858552e+11,9.858552e+11,9.858552e+11,9.858552e+11,2.0,-1.0,0.064516,0.038462
2024-09-26,53.0,9.799228e+11,9.799228e+11,9.799228e+11,9.799228e+11,0.0,-2.0,80.0,9.799228e+11,9.799228e+11,9.799228e+11,9.799228e+11,7.0,-16.0,-0.037736,-0.112500
2024-09-25,57.0,9.755802e+11,9.755802e+11,9.755802e+11,9.755802e+11,3.0,0.0,67.0,9.755802e+11,9.755802e+11,9.755802e+11,9.755802e+11,3.0,-15.0,0.052632,-0.179104
2024-09-24,13.0,9.807434e+11,9.807434e+11,9.807434e+11,9.807434e+11,0.0,-2.0,44.0,9.807434e+11,9.807434e+11,9.807434e+11,9.807434e+11,4.0,-2.0,-0.153846,0.045455


In [ ]:
# Import the Costco data
costco = pd.read_excel("../Sentiment_Data/COST.xlsx")

# Ensure 'Date' is in datetime format
costco['Date'] = pd.to_datetime(costco['Date'], errors='coerce')

# Set 'Date' as the index
costco.set_index('Date', inplace=True)

# Calculate sentiment score for News
costco['News Sentiment Score'] = (costco['News Positive Sentiment Count'] + costco['News Negative Sentiment Count']) / costco['News Publication Count']

# Calculate sentiment score for Twitter
costco['Twitter Sentiment Score'] = (costco['Twitter Positive Sentiment Count'] + costco['Twitter Negative Sentiment Count']) / costco['Twitter Publication Count']

# Display the dataframe with the sentiment scores
costco.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,38.0,3.930254e+11,3.930254e+11,3.930254e+11,3.930254e+11,0.0,0.0,8.0,3.930254e+11,3.930254e+11,3.930254e+11,3.930254e+11,2.0,-1.0,0.000000,0.125000
2024-09-27,355.0,3.926264e+11,3.926264e+11,3.926264e+11,3.926264e+11,36.0,-52.0,132.0,3.926264e+11,3.926264e+11,3.926264e+11,3.926264e+11,27.0,-6.0,-0.045070,0.159091
2024-09-26,90.0,3.996399e+11,3.996399e+11,3.996399e+11,3.996399e+11,20.0,-3.0,32.0,3.996399e+11,3.996399e+11,3.996399e+11,3.996399e+11,6.0,-3.0,0.188889,0.093750
2024-09-25,71.0,4.027344e+11,4.027344e+11,4.027344e+11,4.027344e+11,2.0,-10.0,19.0,4.027344e+11,4.027344e+11,4.027344e+11,4.027344e+11,1.0,-3.0,-0.112676,-0.105263
2024-09-24,120.0,3.996843e+11,3.996843e+11,3.996843e+11,3.996843e+11,10.0,-5.0,21.0,3.996843e+11,3.996843e+11,3.996843e+11,3.996843e+11,1.0,-1.0,0.041667,0.000000


In [ ]:
# Import the Costco data
cisco = pd.read_excel("../Sentiment_Data/CSCO.xlsx")

# Ensure 'Date' is in datetime format
cisco['Date'] = pd.to_datetime(cisco['Date'], errors='coerce')

# Set 'Date' as the index
cisco.set_index('Date', inplace=True)

# Calculate sentiment score for News
cisco['News Sentiment Score'] = (cisco['News Positive Sentiment Count'] + cisco['News Negative Sentiment Count']) / cisco['News Publication Count']

# Calculate sentiment score for Twitter
cisco['Twitter Sentiment Score'] = (cisco['Twitter Positive Sentiment Count'] + cisco['Twitter Negative Sentiment Count']) / cisco['Twitter Publication Count']

# Display the dataframe with the sentiment scores
cisco.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,474.0,2.049678e+12,2.049678e+12,2.049678e+12,2.049678e+12,0.0,-35.0,410.0,2.049678e+12,2.049678e+12,2.049678e+12,2.049678e+12,9.0,-29.0,-0.073840,-0.048780
2024-09-27,808.0,2.026289e+12,2.026289e+12,2.026289e+12,2.026289e+12,6.0,-9.0,632.0,2.026289e+12,2.026289e+12,2.026289e+12,2.026289e+12,8.0,-40.0,-0.003713,-0.050633
2024-09-26,1318.0,2.009826e+12,2.009826e+12,2.009826e+12,2.009826e+12,3.0,-68.0,737.0,2.009826e+12,2.009826e+12,2.009826e+12,2.009826e+12,20.0,-73.0,-0.049317,-0.071913
2024-09-25,899.0,1.996969e+12,1.996969e+12,1.996969e+12,1.996969e+12,6.0,-13.0,733.0,1.996969e+12,1.996969e+12,1.996969e+12,1.996969e+12,9.0,-17.0,-0.007786,-0.010914
2024-09-24,715.0,2.005914e+12,2.005914e+12,2.005914e+12,2.005914e+12,4.0,-15.0,710.0,2.005914e+12,2.005914e+12,2.005914e+12,2.005914e+12,15.0,-34.0,-0.015385,-0.026761


In [ ]:
# Import the ALPHABET data
googl = pd.read_excel("../Sentiment_Data/GOOGL.xlsx")

# Ensure 'Date' is in datetime format
googl['Date'] = pd.to_datetime(googl['Date'], errors='coerce')

# Set 'Date' as the index
googl.set_index('Date', inplace=True)

# Calculate sentiment score for News
googl['News Sentiment Score'] = (googl['News Positive Sentiment Count'] + googl['News Negative Sentiment Count']) / googl['News Publication Count']

# Calculate sentiment score for Twitter
googl['Twitter Sentiment Score'] = (googl['Twitter Positive Sentiment Count'] + googl['Twitter Negative Sentiment Count']) / googl['Twitter Publication Count']

# Display the dataframe with the sentiment scores
googl.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,474.0,2.049678e+12,2.049678e+12,2.049678e+12,2.049678e+12,0.0,-35.0,410.0,2.049678e+12,2.049678e+12,2.049678e+12,2.049678e+12,9.0,-29.0,-0.073840,-0.048780
2024-09-27,808.0,2.026289e+12,2.026289e+12,2.026289e+12,2.026289e+12,6.0,-9.0,632.0,2.026289e+12,2.026289e+12,2.026289e+12,2.026289e+12,8.0,-40.0,-0.003713,-0.050633
2024-09-26,1318.0,2.009826e+12,2.009826e+12,2.009826e+12,2.009826e+12,3.0,-68.0,737.0,2.009826e+12,2.009826e+12,2.009826e+12,2.009826e+12,20.0,-73.0,-0.049317,-0.071913
2024-09-25,899.0,1.996969e+12,1.996969e+12,1.996969e+12,1.996969e+12,6.0,-13.0,733.0,1.996969e+12,1.996969e+12,1.996969e+12,1.996969e+12,9.0,-17.0,-0.007786,-0.010914
2024-09-24,715.0,2.005914e+12,2.005914e+12,2.005914e+12,2.005914e+12,4.0,-15.0,710.0,2.005914e+12,2.005914e+12,2.005914e+12,2.005914e+12,15.0,-34.0,-0.015385,-0.026761


In [ ]:
# Import the JP Morgan Chase data
jpmorgan = pd.read_excel("../Sentiment_Data/JPM.xlsx")

# Ensure 'Date' is in datetime format
jpmorgan['Date'] = pd.to_datetime(jpmorgan['Date'], errors='coerce')

# Set 'Date' as the index
jpmorgan.set_index('Date', inplace=True)

# Calculate sentiment score for News
jpmorgan['News Sentiment Score'] = (jpmorgan['News Positive Sentiment Count'] + jpmorgan['News Negative Sentiment Count']) / jpmorgan['News Publication Count']

# Calculate sentiment score for Twitter
jpmorgan['Twitter Sentiment Score'] = (jpmorgan['Twitter Positive Sentiment Count'] + jpmorgan['Twitter Negative Sentiment Count']) / jpmorgan['Twitter Publication Count']

# Display the dataframe with the sentiment scores
jpmorgan.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,245.0,5.999314e+11,5.999314e+11,5.999314e+11,5.999314e+11,1.0,-1.0,53.0,5.999314e+11,5.999314e+11,5.999314e+11,5.999314e+11,1.0,-2.0,0.000000,-0.018868
2024-09-27,364.0,5.989072e+11,5.989072e+11,5.989072e+11,5.989072e+11,1.0,-4.0,80.0,5.989072e+11,5.989072e+11,5.989072e+11,5.989072e+11,10.0,-1.0,-0.008242,0.112500
2024-09-26,491.0,5.968586e+11,5.968586e+11,5.968586e+11,5.968586e+11,0.0,-3.0,156.0,5.968586e+11,5.968586e+11,5.968586e+11,5.968586e+11,9.0,-3.0,-0.006110,0.038462
2024-09-25,539.0,5.980252e+11,5.980252e+11,5.980252e+11,5.980252e+11,1.0,-1.0,235.0,5.980252e+11,5.980252e+11,5.980252e+11,5.980252e+11,2.0,-10.0,0.000000,-0.034043
2024-09-24,439.0,6.020084e+11,6.020084e+11,6.020084e+11,6.020084e+11,3.0,-3.0,236.0,6.020084e+11,6.020084e+11,6.020084e+11,6.020084e+11,13.0,-8.0,0.000000,0.021186


In [ ]:
# Import the Eli Lilly data
lilly = pd.read_excel("../Sentiment_Data/LLY.xlsx")

# Ensure 'Date' is in datetime format
lilly['Date'] = pd.to_datetime(lilly['Date'], errors='coerce')

# Set 'Date' as the index
lilly.set_index('Date', inplace=True)

# Calculate sentiment score for News
lilly['News Sentiment Score'] = (lilly['News Positive Sentiment Count'] + lilly['News Negative Sentiment Count']) / lilly['News Publication Count']

# Calculate sentiment score for Twitter
lilly['Twitter Sentiment Score'] = (lilly['Twitter Positive Sentiment Count'] + lilly['Twitter Negative Sentiment Count']) / lilly['Twitter Publication Count']

# Display the dataframe with the sentiment scores
lilly.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,41.0,8.420202e+11,8.420202e+11,8.420202e+11,8.420202e+11,0.0,0.0,15.0,8.420202e+11,8.420202e+11,8.420202e+11,8.420202e+11,1.0,-2.0,0.000000,-0.066667
2024-09-27,41.0,8.342743e+11,8.342743e+11,8.342743e+11,8.342743e+11,0.0,-3.0,22.0,8.342743e+11,8.342743e+11,8.342743e+11,8.342743e+11,3.0,-3.0,-0.073171,0.000000
2024-09-26,57.0,8.642412e+11,8.642412e+11,8.642412e+11,8.642412e+11,4.0,-2.0,29.0,8.642412e+11,8.642412e+11,8.642412e+11,8.642412e+11,6.0,-2.0,0.035088,0.137931
2024-09-25,99.0,8.787256e+11,8.787256e+11,8.787256e+11,8.787256e+11,3.0,-3.0,33.0,8.787256e+11,8.787256e+11,8.787256e+11,8.787256e+11,3.0,-1.0,0.000000,0.060606
2024-09-24,86.0,8.786021e+11,8.786021e+11,8.786021e+11,8.786021e+11,2.0,-2.0,29.0,8.786021e+11,8.786021e+11,8.786021e+11,8.786021e+11,3.0,0.0,0.000000,0.103448


In [ ]:
# Import the Meta data
meta = pd.read_excel("../Sentiment_Data/META.xlsx")

# Ensure 'Date' is in datetime format
meta['Date'] = pd.to_datetime(meta['Date'], errors='coerce')

# Set 'Date' as the index
meta.set_index('Date', inplace=True)

# Calculate sentiment score for News
meta['News Sentiment Score'] = (meta['News Positive Sentiment Count'] + meta['News Negative Sentiment Count']) / meta['News Publication Count']

# Calculate sentiment score for Twitter
meta['Twitter Sentiment Score'] = (meta['Twitter Positive Sentiment Count'] + meta['Twitter Negative Sentiment Count']) / meta['Twitter Publication Count']

# Display the dataframe with the sentiment scores
meta.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,354.0,1.448342e+12,1.448342e+12,1.448342e+12,1.448342e+12,21.0,-15.0,117.0,1.448342e+12,1.448342e+12,1.448342e+12,1.448342e+12,4.0,-6.0,0.016949,-0.017094
2024-09-27,770.0,1.435489e+12,1.435489e+12,1.435489e+12,1.435489e+12,13.0,-29.0,245.0,1.435489e+12,1.435489e+12,1.435489e+12,1.435489e+12,6.0,-3.0,-0.020779,0.012245
2024-09-26,1525.0,1.436704e+12,1.436704e+12,1.436704e+12,1.436704e+12,25.0,-11.0,569.0,1.436704e+12,1.436704e+12,1.436704e+12,1.436704e+12,19.0,-8.0,0.009180,0.019332
2024-09-25,525.0,1.437893e+12,1.437893e+12,1.437893e+12,1.437893e+12,6.0,-7.0,211.0,1.437893e+12,1.437893e+12,1.437893e+12,1.437893e+12,1.0,-5.0,-0.001905,-0.018957
2024-09-24,408.0,1.425293e+12,1.425293e+12,1.425293e+12,1.425293e+12,9.0,-5.0,169.0,1.425293e+12,1.425293e+12,1.425293e+12,1.425293e+12,5.0,-10.0,0.009804,-0.029586


In [ ]:
# Import the Microsoft data
msoft = pd.read_excel("../Sentiment_Data/MSFT.xlsx")

# Ensure 'Date' is in datetime format
msoft['Date'] = pd.to_datetime(msoft['Date'], errors='coerce')

# Set 'Date' as the index
msoft.set_index('Date', inplace=True)

# Calculate sentiment score for News
msoft['News Sentiment Score'] = (msoft['News Positive Sentiment Count'] + msoft['News Negative Sentiment Count']) / msoft['News Publication Count']

# Calculate sentiment score for Twitter
msoft['Twitter Sentiment Score'] = (msoft['Twitter Positive Sentiment Count'] + msoft['Twitter Negative Sentiment Count']) / msoft['Twitter Publication Count']

# Display the dataframe with the sentiment scores
msoft.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,476.0,3.198437e+12,3.198437e+12,3.198437e+12,3.198437e+12,0.0,-3.0,237.0,3.198437e+12,3.198437e+12,3.198437e+12,3.198437e+12,8.0,-15.0,-0.006303,-0.029536
2024-09-27,689.0,3.181489e+12,3.181489e+12,3.181489e+12,3.181489e+12,0.0,-10.0,534.0,3.181489e+12,3.181489e+12,3.181489e+12,3.181489e+12,11.0,-29.0,-0.014514,-0.033708
2024-09-26,1451.0,3.205944e+12,3.205944e+12,3.205944e+12,3.205944e+12,7.0,-70.0,845.0,3.205944e+12,3.205944e+12,3.205944e+12,3.205944e+12,20.0,-120.0,-0.043418,-0.118343
2024-09-25,922.0,3.211890e+12,3.211890e+12,3.211890e+12,3.211890e+12,2.0,-8.0,731.0,3.211890e+12,3.211890e+12,3.211890e+12,3.211890e+12,28.0,-20.0,-0.006508,0.010944
2024-09-24,787.0,3.190037e+12,3.190037e+12,3.190037e+12,3.190037e+12,2.0,-19.0,728.0,3.190037e+12,3.190037e+12,3.190037e+12,3.190037e+12,19.0,-52.0,-0.021601,-0.045330


In [ ]:
# Import the Netflix data
netflix = pd.read_excel("../Sentiment_Data/NFLX.xlsx")

# Ensure 'Date' is in datetime format
netflix['Date'] = pd.to_datetime(netflix['Date'], errors='coerce')

# Set 'Date' as the index
netflix.set_index('Date', inplace=True)

# Calculate sentiment score for News
netflix['News Sentiment Score'] = (netflix['News Positive Sentiment Count'] + netflix['News Negative Sentiment Count']) / netflix['News Publication Count']

# Calculate sentiment score for Twitter
netflix['Twitter Sentiment Score'] = (netflix['Twitter Positive Sentiment Count'] + netflix['Twitter Negative Sentiment Count']) / netflix['Twitter Publication Count']

# Display the dataframe with the sentiment scores
netflix.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,8.0,1.740613e+11,1.740613e+11,1.740613e+11,1.740613e+11,0.0,0.0,1.0,1.740613e+11,1.740613e+11,1.740613e+11,1.740613e+11,0.0,0.0,0.000000,0.000000
2024-09-27,77.0,1.735091e+11,1.735091e+11,1.735091e+11,1.735091e+11,5.0,0.0,17.0,1.735091e+11,1.735091e+11,1.735091e+11,1.735091e+11,6.0,0.0,0.064935,0.352941
2024-09-26,21.0,1.784984e+11,1.784984e+11,1.784984e+11,1.784984e+11,0.0,-4.0,5.0,1.784984e+11,1.784984e+11,1.784984e+11,1.784984e+11,1.0,0.0,-0.190476,0.200000
2024-09-25,28.0,1.779125e+11,1.779125e+11,1.779125e+11,1.779125e+11,1.0,-4.0,6.0,1.779125e+11,1.779125e+11,1.779125e+11,1.779125e+11,2.0,0.0,-0.107143,0.333333
2024-09-24,18.0,1.788823e+11,1.788823e+11,1.788823e+11,1.788823e+11,1.0,-3.0,2.0,1.788823e+11,1.788823e+11,1.788823e+11,1.788823e+11,0.0,0.0,-0.111111,0.000000


In [ ]:
# Import the Nvidia data
nvidia = pd.read_excel("../Sentiment_Data/NVDA.xlsx")

# Ensure 'Date' is in datetime format
nvidia['Date'] = pd.to_datetime(nvidia['Date'], errors='coerce')

# Set 'Date' as the index
nvidia.set_index('Date', inplace=True)

# Calculate sentiment score for News
nvidia['News Sentiment Score'] = (nvidia['News Positive Sentiment Count'] + nvidia['News Negative Sentiment Count']) / nvidia['News Publication Count']

# Calculate sentiment score for Twitter
nvidia['Twitter Sentiment Score'] = (nvidia['Twitter Positive Sentiment Count'] + nvidia['Twitter Negative Sentiment Count']) / nvidia['Twitter Publication Count']

# Display the dataframe with the sentiment scores
nvidia.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,513.0,2.978923e+12,2.978923e+12,2.978923e+12,2.978923e+12,6.0,-12.0,863.0,2.978923e+12,2.978923e+12,2.978923e+12,2.978923e+12,36.0,-29.0,-0.011696,0.008111
2024-09-27,835.0,2.977942e+12,2.977942e+12,2.977942e+12,2.977942e+12,28.0,-5.0,2793.0,2.977942e+12,2.977942e+12,2.977942e+12,2.977942e+12,114.0,-67.0,0.027545,0.016828
2024-09-26,1003.0,3.042701e+12,3.042701e+12,3.042701e+12,3.042701e+12,44.0,-8.0,3606.0,3.042701e+12,3.042701e+12,3.042701e+12,3.042701e+12,244.0,-54.0,0.035892,0.052690
2024-09-25,1075.0,3.029700e+12,3.029700e+12,3.029700e+12,3.029700e+12,54.0,-4.0,4066.0,3.029700e+12,3.029700e+12,3.029700e+12,3.029700e+12,248.0,-97.0,0.046512,0.037137
2024-09-24,621.0,2.964941e+12,2.964941e+12,2.964941e+12,2.964941e+12,22.0,-2.0,1930.0,2.964941e+12,2.964941e+12,2.964941e+12,2.964941e+12,73.0,-49.0,0.032206,0.012435


In [ ]:
# Import the ORACLE data
oracle = pd.read_excel("../Sentiment_Data/ORCL.xlsx")

# Ensure 'Date' is in datetime format
oracle['Date'] = pd.to_datetime(oracle['Date'], errors='coerce')

# Set 'Date' as the index
oracle.set_index('Date', inplace=True)

# Calculate sentiment score for News
oracle['News Sentiment Score'] = (oracle['News Positive Sentiment Count'] + oracle['News Negative Sentiment Count']) / oracle['News Publication Count']

# Calculate sentiment score for Twitter
oracle['Twitter Sentiment Score'] = (oracle['Twitter Positive Sentiment Count'] + oracle['Twitter Negative Sentiment Count']) / oracle['Twitter Publication Count']

# Display the dataframe with the sentiment scores
oracle.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,45.0,4.721891e+11,4.721891e+11,4.721891e+11,4.721891e+11,6.0,0.0,5.0,4.721891e+11,4.721891e+11,4.721891e+11,4.721891e+11,1.0,0.0,0.133333,0.200000
2024-09-27,77.0,4.675892e+11,4.675892e+11,4.675892e+11,4.675892e+11,10.0,-2.0,19.0,4.675892e+11,4.675892e+11,4.675892e+11,4.675892e+11,1.0,0.0,0.103896,0.052632
2024-09-26,144.0,4.658157e+11,4.658157e+11,4.658157e+11,4.658157e+11,6.0,-5.0,46.0,4.658157e+11,4.658157e+11,4.658157e+11,4.658157e+11,3.0,-2.0,0.006944,0.021739
2024-09-25,73.0,4.607565e+11,4.607565e+11,4.607565e+11,4.607565e+11,2.0,-10.0,34.0,4.607565e+11,4.607565e+11,4.607565e+11,4.607565e+11,2.0,0.0,-0.109589,0.058824
2024-09-24,78.0,4.594265e+11,4.594265e+11,4.594265e+11,4.594265e+11,0.0,-10.0,38.0,4.594265e+11,4.594265e+11,4.594265e+11,4.594265e+11,1.0,0.0,-0.128205,0.026316


In [ ]:
# Import the Pepsi data
pepsi = pd.read_excel("../Sentiment_Data/PEP.xlsx")

# Ensure 'Date' is in datetime format
pepsi['Date'] = pd.to_datetime(pepsi['Date'], errors='coerce')

# Set 'Date' as the index
pepsi.set_index('Date', inplace=True)

# Calculate sentiment score for News
pepsi['News Sentiment Score'] = (pepsi['News Positive Sentiment Count'] + pepsi['News Negative Sentiment Count']) / pepsi['News Publication Count']

# Calculate sentiment score for Twitter
pepsi['Twitter Sentiment Score'] = (pepsi['Twitter Positive Sentiment Count'] + pepsi['Twitter Negative Sentiment Count']) / pepsi['Twitter Publication Count']

# Display the dataframe with the sentiment scores
pepsi.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,45.0,2.335760e+11,2.335760e+11,2.335760e+11,2.335760e+11,0.0,0.0,15.0,2.335760e+11,2.335760e+11,2.335760e+11,2.335760e+11,1.0,0.0,0.000000,0.066667
2024-09-27,58.0,2.335073e+11,2.335073e+11,2.335073e+11,2.335073e+11,2.0,-5.0,21.0,2.335073e+11,2.335073e+11,2.335073e+11,2.335073e+11,2.0,-1.0,-0.051724,0.047619
2024-09-26,56.0,2.330815e+11,2.330815e+11,2.330815e+11,2.330815e+11,0.0,-3.0,25.0,2.330815e+11,2.330815e+11,2.330815e+11,2.330815e+11,0.0,-1.0,-0.053571,-0.040000
2024-09-25,82.0,2.326832e+11,2.326832e+11,2.326832e+11,2.326832e+11,0.0,-2.0,24.0,2.326832e+11,2.326832e+11,2.326832e+11,2.326832e+11,0.0,0.0,-0.024390,0.000000
2024-09-24,61.0,2.333974e+11,2.333974e+11,2.333974e+11,2.333974e+11,2.0,-1.0,15.0,2.333974e+11,2.333974e+11,2.333974e+11,2.333974e+11,0.0,0.0,0.016393,0.000000


In [ ]:
# Import the Procter & Gamble data
pg = pd.read_excel("../Sentiment_Data/PG.xlsx")

# Ensure 'Date' is in datetime format
pg['Date'] = pd.to_datetime(pg['Date'], errors='coerce')

# Set 'Date' as the index
pg.set_index('Date', inplace=True)

# Calculate sentiment score for News
pg['News Sentiment Score'] = (pg['News Positive Sentiment Count'] + pg['News Negative Sentiment Count']) / pg['News Publication Count']

# Calculate sentiment score for Twitter
pg['Twitter Sentiment Score'] = (pg['Twitter Positive Sentiment Count'] + pg['Twitter Negative Sentiment Count']) / pg['Twitter Publication Count']

# Display the dataframe with the sentiment scores
pg.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,36.0,4.069694e+11,4.069694e+11,4.069694e+11,4.069694e+11,0.0,-2.0,7.0,4.069694e+11,4.069694e+11,4.069694e+11,4.069694e+11,1.0,-3.0,-0.055556,-0.285714
2024-09-27,35.0,4.077918e+11,4.077918e+11,4.077918e+11,4.077918e+11,12.0,0.0,5.0,4.077918e+11,4.077918e+11,4.077918e+11,4.077918e+11,1.0,0.0,0.342857,0.200000
2024-09-26,40.0,4.069929e+11,4.069929e+11,4.069929e+11,4.069929e+11,0.0,-11.0,9.0,4.069929e+11,4.069929e+11,4.069929e+11,4.069929e+11,2.0,0.0,-0.275000,0.222222
2024-09-25,74.0,4.047607e+11,4.047607e+11,4.047607e+11,4.047607e+11,0.0,-14.0,19.0,4.047607e+11,4.047607e+11,4.047607e+11,4.047607e+11,2.0,-2.0,-0.189189,0.000000
2024-09-24,86.0,4.070634e+11,4.070634e+11,4.070634e+11,4.070634e+11,0.0,-11.0,10.0,4.070634e+11,4.070634e+11,4.070634e+11,4.070634e+11,0.0,0.0,-0.127907,0.000000


In [ ]:
# Import the T Mobil US data
tmobil = pd.read_excel("../Sentiment_Data/TMUS.xlsx")

# Ensure 'Date' is in datetime format
tmobil['Date'] = pd.to_datetime(tmobil['Date'], errors='coerce')

# Set 'Date' as the index
tmobil.set_index('Date', inplace=True)

# Calculate sentiment score for News
tmobil['News Sentiment Score'] = (tmobil['News Positive Sentiment Count'] + tmobil['News Negative Sentiment Count']) / tmobil['News Publication Count']

# Calculate sentiment score for Twitter
tmobil['Twitter Sentiment Score'] = (tmobil['Twitter Positive Sentiment Count'] + tmobil['Twitter Negative Sentiment Count']) / tmobil['Twitter Publication Count']

# Display the dataframe with the sentiment scores
tmobil.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,13.0,1.886079e+11,1.886079e+11,1.886079e+11,1.886079e+11,0.0,0.0,2.0,1.886079e+11,1.886079e+11,1.886079e+11,1.886079e+11,0.0,0.0,0.000000,0.000000
2024-09-27,24.0,1.909544e+11,1.909544e+11,1.909544e+11,1.909544e+11,2.0,0.0,9.0,1.909544e+11,1.909544e+11,1.909544e+11,1.909544e+11,0.0,-1.0,0.083333,-0.111111
2024-09-26,26.0,1.910640e+11,1.910640e+11,1.910640e+11,1.910640e+11,1.0,-1.0,4.0,1.910640e+11,1.910640e+11,1.910640e+11,1.910640e+11,0.0,0.0,0.000000,0.000000
2024-09-25,41.0,1.872292e+11,1.872292e+11,1.872292e+11,1.872292e+11,2.0,-1.0,5.0,1.872292e+11,1.872292e+11,1.872292e+11,1.872292e+11,0.0,0.0,0.024390,0.000000
2024-09-24,32.0,1.871196e+11,1.871196e+11,1.871196e+11,1.871196e+11,2.0,0.0,9.0,1.871196e+11,1.871196e+11,1.871196e+11,1.871196e+11,0.0,-1.0,0.062500,-0.111111


In [ ]:
# Import the Tesla data
tesla = pd.read_excel("../Sentiment_Data/TSLA.xlsx")

# Ensure 'Date' is in datetime format
tesla['Date'] = pd.to_datetime(tesla['Date'], errors='coerce')

# Set 'Date' as the index
tesla.set_index('Date', inplace=True)

# Calculate sentiment score for News
tesla['News Sentiment Score'] = (tesla['News Positive Sentiment Count'] + tesla['News Negative Sentiment Count']) / tesla['News Publication Count']

# Calculate sentiment score for Twitter
tesla['Twitter Sentiment Score'] = (tesla['Twitter Positive Sentiment Count'] + tesla['Twitter Negative Sentiment Count']) / tesla['Twitter Publication Count']

# Display the dataframe with the sentiment scores
tesla.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,345.0,8.358138e+11,8.358138e+11,8.358138e+11,8.358138e+11,8.0,-1.0,1017.0,8.358138e+11,8.358138e+11,8.358138e+11,8.358138e+11,72.0,-63.0,0.020290,0.008850
2024-09-27,487.0,8.320761e+11,8.320761e+11,8.320761e+11,8.320761e+11,8.0,-9.0,1252.0,8.320761e+11,8.320761e+11,8.320761e+11,8.320761e+11,123.0,-76.0,-0.002053,0.037540
2024-09-26,656.0,8.121415e+11,8.121415e+11,8.121415e+11,8.121415e+11,27.0,-4.0,1583.0,8.121415e+11,8.121415e+11,8.121415e+11,8.121415e+11,176.0,-90.0,0.035061,0.054327
2024-09-25,548.0,8.210865e+11,8.210865e+11,8.210865e+11,8.210865e+11,25.0,-4.0,1457.0,8.210865e+11,8.210865e+11,8.210865e+11,8.210865e+11,178.0,-84.0,0.038321,0.064516
2024-09-24,714.0,8.123012e+11,8.123012e+11,8.123012e+11,8.123012e+11,55.0,-5.0,1964.0,8.123012e+11,8.123012e+11,8.123012e+11,8.123012e+11,290.0,-97.0,0.070028,0.098269


In [ ]:
# Import the UnitedHealth Group data
uhealth = pd.read_excel("../Sentiment_Data/UNH.xlsx")

# Ensure 'Date' is in datetime format
uhealth['Date'] = pd.to_datetime(uhealth['Date'], errors='coerce')

# Set 'Date' as the index
uhealth.set_index('Date', inplace=True)

# Calculate sentiment score for News
uhealth['News Sentiment Score'] = (uhealth['News Positive Sentiment Count'] + uhealth['News Negative Sentiment Count']) / uhealth['News Publication Count']

# Calculate sentiment score for Twitter
uhealth['Twitter Sentiment Score'] = (uhealth['Twitter Positive Sentiment Count'] + uhealth['Twitter Negative Sentiment Count']) / uhealth['Twitter Publication Count']

# Display the dataframe with the sentiment scores
uhealth.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,18.0,5.399041e+11,5.399041e+11,5.399041e+11,5.399041e+11,0.0,0.0,1.0,5.399041e+11,5.399041e+11,5.399041e+11,5.399041e+11,0.0,-1.0,0.000000,-1.000000
2024-09-27,57.0,5.372909e+11,5.372909e+11,5.372909e+11,5.372909e+11,0.0,-1.0,11.0,5.372909e+11,5.372909e+11,5.372909e+11,5.372909e+11,1.0,-2.0,-0.017544,-0.090909
2024-09-26,26.0,5.307900e+11,5.307900e+11,5.307900e+11,5.307900e+11,0.0,-3.0,9.0,5.307900e+11,5.307900e+11,5.307900e+11,5.307900e+11,0.0,0.0,-0.115385,0.000000
2024-09-25,21.0,5.321751e+11,5.321751e+11,5.321751e+11,5.321751e+11,3.0,0.0,10.0,5.321751e+11,5.321751e+11,5.321751e+11,5.321751e+11,0.0,0.0,0.142857,0.000000
2024-09-24,35.0,5.311409e+11,5.311409e+11,5.311409e+11,5.311409e+11,1.0,-2.0,10.0,5.311409e+11,5.311409e+11,5.311409e+11,5.311409e+11,0.0,-2.0,-0.028571,-0.200000


In [ ]:
# Import the Visa data
visa = pd.read_excel("../Sentiment_Data/V.xlsx")

# Ensure 'Date' is in datetime format
visa['Date'] = pd.to_datetime(visa['Date'], errors='coerce')

# Set 'Date' as the index
visa.set_index('Date', inplace=True)

# Calculate sentiment score for News
visa['News Sentiment Score'] = (visa['News Positive Sentiment Count'] + visa['News Negative Sentiment Count']) / visa['News Publication Count']

# Calculate sentiment score for Twitter
visa['Twitter Sentiment Score'] = (visa['Twitter Positive Sentiment Count'] + visa['Twitter Negative Sentiment Count']) / visa['Twitter Publication Count']

# Display the dataframe with the sentiment scores
visa.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,19.0,5.572310e+11,5.572310e+11,5.572310e+11,5.572310e+11,0.0,-2.0,34.0,5.572310e+11,5.572310e+11,5.572310e+11,5.572310e+11,4.0,-9.0,-0.105263,-0.147059
2024-09-27,110.0,5.576769e+11,5.576769e+11,5.576769e+11,5.576769e+11,3.0,-10.0,128.0,5.576769e+11,5.576769e+11,5.576769e+11,5.576769e+11,3.0,-21.0,-0.063636,-0.140625
2024-09-26,119.0,5.506241e+11,5.506241e+11,5.506241e+11,5.506241e+11,0.0,-39.0,200.0,5.506241e+11,5.506241e+11,5.506241e+11,5.506241e+11,3.0,-24.0,-0.327731,-0.105000
2024-09-25,427.0,5.464492e+11,5.464492e+11,5.464492e+11,5.464492e+11,1.0,-185.0,402.0,5.464492e+11,5.464492e+11,5.464492e+11,5.464492e+11,8.0,-140.0,-0.430913,-0.328358
2024-09-24,96.0,5.528331e+11,5.528331e+11,5.528331e+11,5.528331e+11,1.0,-29.0,96.0,5.528331e+11,5.528331e+11,5.528331e+11,5.528331e+11,4.0,-39.0,-0.291667,-0.364583


In [ ]:
# Import the Walmart data
walmart = pd.read_excel("../Sentiment_Data/WMT.xlsx")

# Ensure 'Date' is in datetime format
walmart['Date'] = pd.to_datetime(walmart['Date'], errors='coerce')

# Set 'Date' as the index
walmart.set_index('Date', inplace=True)

# Calculate sentiment score for News
walmart['News Sentiment Score'] = (walmart['News Positive Sentiment Count'] + walmart['News Negative Sentiment Count']) / walmart['News Publication Count']

# Calculate sentiment score for Twitter
walmart['Twitter Sentiment Score'] = (walmart['Twitter Positive Sentiment Count'] + walmart['Twitter Negative Sentiment Count']) / walmart['Twitter Publication Count']

# Display the dataframe with the sentiment scores
walmart.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,125.0,6.490888e+11,6.490888e+11,6.490888e+11,6.490888e+11,1.0,-1.0,18.0,6.490888e+11,6.490888e+11,6.490888e+11,6.490888e+11,5.0,-2.0,0.000000,0.166667
2024-09-27,178.0,6.412916e+11,6.412916e+11,6.412916e+11,6.412916e+11,2.0,-1.0,55.0,6.412916e+11,6.412916e+11,6.412916e+11,6.412916e+11,2.0,-2.0,0.005618,0.000000
2024-09-26,201.0,6.424170e+11,6.424170e+11,6.424170e+11,6.424170e+11,2.0,0.0,59.0,6.424170e+11,6.424170e+11,6.424170e+11,6.424170e+11,6.0,-1.0,0.009950,0.084746
2024-09-25,214.0,6.543137e+11,6.543137e+11,6.543137e+11,6.543137e+11,13.0,0.0,71.0,6.543137e+11,6.543137e+11,6.543137e+11,6.543137e+11,9.0,-2.0,0.060748,0.098592
2024-09-24,242.0,6.484457e+11,6.484457e+11,6.484457e+11,6.484457e+11,6.0,-1.0,80.0,6.484457e+11,6.484457e+11,6.484457e+11,6.484457e+11,6.0,-2.0,0.020661,0.050000


In [ ]:
# Import the Exxon Mobil  data
xmobile = pd.read_excel("../Sentiment_Data/XOM.xlsx")

# Ensure 'Date' is in datetime format
xmobile['Date'] = pd.to_datetime(xmobile['Date'], errors='coerce')

# Set 'Date' as the index
xmobile.set_index('Date', inplace=True)

# Calculate sentiment score for News
xmobile['News Sentiment Score'] = (xmobile['News Positive Sentiment Count'] + xmobile['News Negative Sentiment Count']) / xmobile['News Publication Count']

# Calculate sentiment score for Twitter
xmobile['Twitter Sentiment Score'] = (xmobile['Twitter Positive Sentiment Count'] + xmobile['Twitter Negative Sentiment Count']) / xmobile['Twitter Publication Count']

# Display the dataframe with the sentiment scores
xmobile.head()

,News Publication Count,(R1) Open,(R1) High,(R1) Low,(R1) Close,News Positive Sentiment Count,News Negative Sentiment Count,Twitter Publication Count,(R1) Open.1,(R1) High.1,(R1) Low.1,(R1) Close.1,Twitter Positive Sentiment Count,Twitter Negative Sentiment Count,News Sentiment Score,Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,
2024-09-30,121.0,5.207881e+11,5.207881e+11,5.207881e+11,5.207881e+11,0.0,-1.0,70.0,5.207881e+11,5.207881e+11,5.207881e+11,5.207881e+11,3.0,-18.0,-0.008264,-0.214286
2024-09-27,143.0,5.145682e+11,5.145682e+11,5.145682e+11,5.145682e+11,0.0,-14.0,177.0,5.145682e+11,5.145682e+11,5.145682e+11,5.145682e+11,9.0,-40.0,-0.097902,-0.175141
2024-09-26,113.0,5.011508e+11,5.011508e+11,5.011508e+11,5.011508e+11,0.0,-7.0,181.0,5.011508e+11,5.011508e+11,5.011508e+11,5.011508e+11,2.0,-45.0,-0.061947,-0.237569
2024-09-25,214.0,5.099032e+11,5.099032e+11,5.099032e+11,5.099032e+11,0.0,-21.0,246.0,5.099032e+11,5.099032e+11,5.099032e+11,5.099032e+11,1.0,-133.0,-0.098131,-0.536585
2024-09-24,416.0,5.200328e+11,5.200328e+11,5.200328e+11,5.200328e+11,6.0,-125.0,497.0,5.200328e+11,5.200328e+11,5.200328e+11,5.200328e+11,4.0,-355.0,-0.286058,-0.706237


## Load all the sentiment data in one data frame

Create a dataframe that contains 15 most influential stocks for Nasdaq 100 and add only the Sentiment data in this dataframe. Then, create another dataframe that contains only 6 columns where each column is the sum of all the stocks.

## Nasdaq 100

In [35]:
# Create a dictionary of dataframes with their respective tickers
stocks = {
    "AAPL": apple,
    "NVDA": nvidia,
    "MSFT": msoft,
    "AMZN": amazon,
    "GOOGL": googl,
    "META": meta,
    "TSLA": tesla,
    "AVGO": broadcom,
    "COST": costco,
    "NFLX": netflix,
    "TMUS": tmobil,
    "ASML": asml,
    "CSCO": cisco,
    "ADBE": adobe,
    "PEP": pepsi  
}

# List of desired columns
selected_columns = [
    "News Sentiment Score",
    "Twitter Sentiment Score",
]

# Create a new DataFrame to store the results
ndx = pd.DataFrame()

for ticker, df in stocks.items():
    # Clean column names: strip spaces
    df.columns = df.columns.str.strip()
    
    # Check if all selected columns exist in the DataFrame
    missing_columns = [col for col in selected_columns if col not in df.columns]
    if missing_columns:
        print(f"{ticker} is missing columns: {missing_columns}. Skipping...")
        continue
    
    # Select and rename columns with the ticker name prefix
    renamed_df = df[selected_columns].add_prefix(f"{ticker} ")
    
    # Concatenate the data
    if ndx.empty:
        ndx = renamed_df
    else:
        ndx = pd.concat([ndx, renamed_df], axis=1)

# Check the combined DataFrame
ndx.head()

,AAPL News Sentiment Score,AAPL Twitter Sentiment Score,NVDA News Sentiment Score,NVDA Twitter Sentiment Score,MSFT News Sentiment Score,MSFT Twitter Sentiment Score,AMZN News Sentiment Score,AMZN Twitter Sentiment Score,GOOGL News Sentiment Score,GOOGL Twitter Sentiment Score,...,TMUS News Sentiment Score,TMUS Twitter Sentiment Score,ASML News Sentiment Score,ASML Twitter Sentiment Score,CSCO News Sentiment Score,CSCO Twitter Sentiment Score,ADBE News Sentiment Score,ADBE Twitter Sentiment Score,PEP News Sentiment Score,PEP Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,0.004086,-0.168911,NaN,NaN,-0.005576,0.000000,0.016807,-0.292929,-0.005168,0.007874,...,NaN,NaN,NaN,NaN,-0.005168,0.007874,NaN,NaN,NaN,NaN
2015-01-05,0.022677,-0.105616,0.123377,NaN,0.023256,-0.010169,NaN,0.194872,0.009238,0.000000,...,NaN,NaN,NaN,NaN,0.009238,0.000000,NaN,NaN,0.094118,NaN
2015-01-06,0.018311,-0.068352,NaN,0.009434,0.043785,0.010517,0.084906,0.030501,0.006747,-0.078540,...,0.007937,NaN,NaN,0.210526,0.006747,-0.078540,0.0625,NaN,0.039216,0.540541
2015-01-07,0.036989,0.005421,NaN,NaN,0.022041,-0.045333,0.055118,-0.200431,0.003236,0.019737,...,0.056180,NaN,-0.024390,-0.038462,0.003236,0.019737,NaN,NaN,NaN,0.090909
2015-01-08,0.039543,0.036997,0.056604,0.000000,0.033175,-0.016447,0.076696,-0.020333,-0.004938,-0.050566,...,NaN,NaN,0.021277,0.000000,-0.004938,-0.050566,NaN,NaN,0.111801,0.098592


In [36]:
# Check for missing values
ndx.isnull().sum()  # this counts the missing values in each column

AAPL News Sentiment Score          4
AAPL Twitter Sentiment Score       3
NVDA News Sentiment Score        111
NVDA Twitter Sentiment Score     121
MSFT News Sentiment Score          6
MSFT Twitter Sentiment Score       3
AMZN News Sentiment Score         17
AMZN Twitter Sentiment Score       3
GOOGL News Sentiment Score        11
GOOGL Twitter Sentiment Score      3
META News Sentiment Score          5
META Twitter Sentiment Score       3
TSLA News Sentiment Score          9
TSLA Twitter Sentiment Score       3
AVGO News Sentiment Score        119
AVGO Twitter Sentiment Score     562
COST News Sentiment Score         77
COST Twitter Sentiment Score     757
NFLX News Sentiment Score        223
NFLX Twitter Sentiment Score     999
TMUS News Sentiment Score        163
TMUS Twitter Sentiment Score     803
ASML News Sentiment Score        167
ASML Twitter Sentiment Score      61
CSCO News Sentiment Score         11
CSCO Twitter Sentiment Score       3
ADBE News Sentiment Score        197
A

There is a lot of missing data. Therefore, it is necessary to decide on a methode to handle them. I will impute these values using the median (so I can avoid biases from the mean imputation)

In [37]:
# Replace the missing values with the median
ndx = ndx.fillna(ndx.median())

# Check again to make sure we do not have any missing data
ndx.isnull().sum()  


AAPL News Sentiment Score        0
AAPL Twitter Sentiment Score     0
NVDA News Sentiment Score        0
NVDA Twitter Sentiment Score     0
MSFT News Sentiment Score        0
MSFT Twitter Sentiment Score     0
AMZN News Sentiment Score        0
AMZN Twitter Sentiment Score     0
GOOGL News Sentiment Score       0
GOOGL Twitter Sentiment Score    0
META News Sentiment Score        0
META Twitter Sentiment Score     0
TSLA News Sentiment Score        0
TSLA Twitter Sentiment Score     0
AVGO News Sentiment Score        0
AVGO Twitter Sentiment Score     0
COST News Sentiment Score        0
COST Twitter Sentiment Score     0
NFLX News Sentiment Score        0
NFLX Twitter Sentiment Score     0
TMUS News Sentiment Score        0
TMUS Twitter Sentiment Score     0
ASML News Sentiment Score        0
ASML Twitter Sentiment Score     0
CSCO News Sentiment Score        0
CSCO Twitter Sentiment Score     0
ADBE News Sentiment Score        0
ADBE Twitter Sentiment Score     0
PEP News Sentiment S

## S&P 500

In [38]:
# Create a dictionary of dataframes with their respective tickers
stocks = {
    "AAPL": apple,
    "NVDA": nvidia,
    "MSFT": msoft,
    "AMZN": amazon,
    "GOOGL": googl,
    "META": meta,
    "TSLA": tesla,
    "BRK.B": berkhath,
    "LLY": lilly,
    "AVGO": broadcom,
    "WMT": walmart,
    "JPM": jpmorgan,
    "V": visa,
    "UNH": uhealth,
    "XOM": xmobile  
}

# List of desired columns
selected_columns = [
     "News Sentiment Score",
    "Twitter Sentiment Score",
]


# Create a new DataFrame to store the results
sp = pd.DataFrame()

for ticker, df in stocks.items():
    # Clean column names: strip spaces
    df.columns = df.columns.str.strip()
    
    # Check if all selected columns exist in the DataFrame
    missing_columns = [col for col in selected_columns if col not in df.columns]
    if missing_columns:
        print(f"{ticker} is missing columns: {missing_columns}. Skipping...")
        continue
    
    # Select and rename columns with the ticker name prefix
    renamed_df = df[selected_columns].add_prefix(f"{ticker} ")
    
    # Concatenate the data
    if sp.empty:
        sp = renamed_df
    else:
        sp = pd.concat([sp, renamed_df], axis=1)

# Check the combined DataFrame
sp

,AAPL News Sentiment Score,AAPL Twitter Sentiment Score,NVDA News Sentiment Score,NVDA Twitter Sentiment Score,MSFT News Sentiment Score,MSFT Twitter Sentiment Score,AMZN News Sentiment Score,AMZN Twitter Sentiment Score,GOOGL News Sentiment Score,GOOGL Twitter Sentiment Score,...,WMT News Sentiment Score,WMT Twitter Sentiment Score,JPM News Sentiment Score,JPM Twitter Sentiment Score,V News Sentiment Score,V Twitter Sentiment Score,UNH News Sentiment Score,UNH Twitter Sentiment Score,XOM News Sentiment Score,XOM Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,,,,,,
2024-09-30,-0.005988,-0.043243,-0.011696,0.008111,-0.006303,-0.029536,-0.011976,-0.027273,-0.073840,-0.048780,...,0.000000,0.166667,0.000000,-0.018868,-0.105263,-0.147059,0.000000,-1.000000,-0.008264,-0.214286
2024-09-27,-0.003115,-0.039604,0.027545,0.016828,-0.014514,-0.033708,-0.058480,0.000000,-0.003713,-0.050633,...,0.005618,0.000000,-0.008242,0.112500,-0.063636,-0.140625,-0.017544,-0.090909,-0.097902,-0.175141
2024-09-26,-0.025063,-0.051630,0.035892,0.052690,-0.043418,-0.118343,-0.018692,-0.029289,-0.049317,-0.071913,...,0.009950,0.084746,-0.006110,0.038462,-0.327731,-0.105000,-0.115385,0.000000,-0.061947,-0.237569
2024-09-25,0.007344,-0.046053,0.046512,0.037137,-0.006508,0.010944,0.018868,0.004082,-0.007786,-0.010914,...,0.060748,0.098592,0.000000,-0.034043,-0.430913,-0.328358,0.142857,0.000000,-0.098131,-0.536585
2024-09-24,-0.001019,-0.025000,0.032206,0.012435,-0.021601,-0.045330,0.012232,0.047945,-0.015385,-0.026761,...,0.020661,0.050000,0.000000,0.021186,-0.291667,-0.364583,-0.028571,-0.200000,-0.286058,-0.706237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-08,0.039543,0.036997,0.056604,0.000000,0.033175,-0.016447,0.076696,-0.020333,-0.004938,-0.050566,...,0.043919,0.080321,0.017391,0.036900,NaN,0.000000,0.031250,-0.322581,0.026119,-0.046512
2015-01-07,0.036989,0.005421,NaN,NaN,0.022041,-0.045333,0.055118,-0.200431,0.003236,0.019737,...,-0.003165,-0.040359,0.012585,0.085774,0.071429,NaN,0.000000,0.041667,-0.044983,-0.092391
2015-01-06,0.018311,-0.068352,NaN,0.009434,0.043785,0.010517,0.084906,0.030501,0.006747,-0.078540,...,-0.036415,-0.023077,-0.001471,-0.210454,-0.083333,0.000000,NaN,-0.045455,0.010949,-0.233333


In [39]:
# Check for missing values
sp.isnull().sum()  # this counts the missing values in each column

AAPL News Sentiment Score          3
AAPL Twitter Sentiment Score       2
NVDA News Sentiment Score        110
NVDA Twitter Sentiment Score     120
MSFT News Sentiment Score          5
MSFT Twitter Sentiment Score       2
AMZN News Sentiment Score         16
AMZN Twitter Sentiment Score       2
GOOGL News Sentiment Score        10
GOOGL Twitter Sentiment Score      2
META News Sentiment Score          4
META Twitter Sentiment Score       2
TSLA News Sentiment Score          8
TSLA Twitter Sentiment Score       2
BRK.B News Sentiment Score        82
BRK.B Twitter Sentiment Score    308
LLY News Sentiment Score          81
LLY Twitter Sentiment Score      462
AVGO News Sentiment Score        118
AVGO Twitter Sentiment Score     561
WMT News Sentiment Score          12
WMT Twitter Sentiment Score       14
JPM News Sentiment Score          10
JPM Twitter Sentiment Score       14
V News Sentiment Score           118
V Twitter Sentiment Score        243
UNH News Sentiment Score         161
U

In [40]:
# Replace the missing values with the median
sp = sp.fillna(sp.median())

# Check again to make sure we do not have any missing data
sp.isnull().sum()  

AAPL News Sentiment Score        0
AAPL Twitter Sentiment Score     0
NVDA News Sentiment Score        0
NVDA Twitter Sentiment Score     0
MSFT News Sentiment Score        0
MSFT Twitter Sentiment Score     0
AMZN News Sentiment Score        0
AMZN Twitter Sentiment Score     0
GOOGL News Sentiment Score       0
GOOGL Twitter Sentiment Score    0
META News Sentiment Score        0
META Twitter Sentiment Score     0
TSLA News Sentiment Score        0
TSLA Twitter Sentiment Score     0
BRK.B News Sentiment Score       0
BRK.B Twitter Sentiment Score    0
LLY News Sentiment Score         0
LLY Twitter Sentiment Score      0
AVGO News Sentiment Score        0
AVGO Twitter Sentiment Score     0
WMT News Sentiment Score         0
WMT Twitter Sentiment Score      0
JPM News Sentiment Score         0
JPM Twitter Sentiment Score      0
V News Sentiment Score           0
V Twitter Sentiment Score        0
UNH News Sentiment Score         0
UNH Twitter Sentiment Score      0
XOM News Sentiment S

# Weighted Sentiment Score

To calculate a weighted sentiment score for the any stock index, we would need to determine a set of weights based on the market capitalization of the companies (or their importance in the index) and then compute the weighted average of the sentiment scores.

Weighted Sentiment Score Calculation:

For each index, each company's sentiment score (whether for news or Twitter) is multiplied by its market cap weight, and then these weighted scores are summed to get the overall weighted sentiment score for the S&P.

The formula would be:

$$\text{Weighted Sentiment Score} = \frac{\sum_{i=1}^{n} \left( \text{Weight}_i \times \text{Sentiment Score}_i \right)}{\sum_{i=1}^{n} \text{Weight}_i}$$

where: 

* $\text{Weight}_i$ is the weight of stock 

* $i$ is the index, and 

* $\text{Sentiment Score}_i$ is the sentiment score for stock 

Using market cap as the weight ensures that stocks with larger market values have a bigger impact on the index’s overall sentiment score.
Summing the weighted scores accounts for each stock’s contribution to the index in a balanced manner.

In [41]:
# Weighted Sentiment Score for Nasdaq 100

weights = {
    "AAPL": nasdaq100_companies.loc[0, 'Market Cap Percentage'],
    "NVDA": nasdaq100_companies.loc[1, 'Market Cap Percentage'],
    "MSFT": nasdaq100_companies.loc[2, 'Market Cap Percentage'],
    "AMZN": nasdaq100_companies.loc[3, 'Market Cap Percentage'],
    "GOOGL": nasdaq100_companies.loc[4, 'Market Cap Percentage'],
    "META": nasdaq100_companies.loc[5, 'Market Cap Percentage'],
    "TSLA": nasdaq100_companies.loc[6, 'Market Cap Percentage'],
    "AVGO": nasdaq100_companies.loc[7, 'Market Cap Percentage'],
    "COST": nasdaq100_companies.loc[8, 'Market Cap Percentage'],
    "NFLX": nasdaq100_companies.loc[9, 'Market Cap Percentage'],
    "TMUS": nasdaq100_companies.loc[10, 'Market Cap Percentage'],
    "ASML": nasdaq100_companies.loc[11, 'Market Cap Percentage'],
    "CSCO": nasdaq100_companies.loc[12, 'Market Cap Percentage'],
    "ADBE": nasdaq100_companies.loc[13, 'Market Cap Percentage'],
    "PEP": nasdaq100_companies.loc[14, 'Market Cap Percentage']
}

# Function to calculate weighted sentiment score
def calculate_weighted_sentiment(ndx, weights, sentiment_type):
    weighted_sentiment = []
    
    # Calculate weighted sentiment score for each date
    for index, row in ndx.iterrows():
        weighted_score = 0
        total_weight = 0
        for stock, weight in weights.items():
            sentiment_column = f'{stock} {sentiment_type} Sentiment Score'
            if sentiment_column in ndx.columns:
                weighted_score += row[sentiment_column] * weight
                total_weight += weight
        # Store weighted sentiment for each date
        weighted_sentiment.append(weighted_score / total_weight if total_weight > 0 else 0)
    
    return weighted_sentiment

# Calculate the weighted sentiment scores for News and Twitter
ndx['Weighted News Sentiment Score'] = calculate_weighted_sentiment(ndx, weights, 'News')
ndx['Weighted Twitter Sentiment Score'] = calculate_weighted_sentiment(ndx, weights, 'Twitter')

# Display the results
ndx


,AAPL News Sentiment Score,AAPL Twitter Sentiment Score,NVDA News Sentiment Score,NVDA Twitter Sentiment Score,MSFT News Sentiment Score,MSFT Twitter Sentiment Score,AMZN News Sentiment Score,AMZN Twitter Sentiment Score,GOOGL News Sentiment Score,GOOGL Twitter Sentiment Score,...,ASML News Sentiment Score,ASML Twitter Sentiment Score,CSCO News Sentiment Score,CSCO Twitter Sentiment Score,ADBE News Sentiment Score,ADBE Twitter Sentiment Score,PEP News Sentiment Score,PEP Twitter Sentiment Score,Weighted News Sentiment Score,Weighted Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,0.004086,-0.168911,0.006018,0.029394,-0.005576,0.000000,0.016807,-0.292929,-0.005168,0.007874,...,0.000000,0.017709,-0.005168,0.007874,0.000000,0.032258,0.000000,0.010782,0.003931,-0.049428
2015-01-05,0.022677,-0.105616,0.123377,0.029394,0.023256,-0.010169,0.001076,0.194872,0.009238,0.000000,...,0.000000,0.017709,0.009238,0.000000,0.000000,0.032258,0.094118,0.010782,0.035583,0.007654
2015-01-06,0.018311,-0.068352,0.006018,0.009434,0.043785,0.010517,0.084906,0.030501,0.006747,-0.078540,...,0.000000,0.210526,0.006747,-0.078540,0.062500,0.032258,0.039216,0.540541,0.025765,-0.003190
2015-01-07,0.036989,0.005421,0.006018,0.029394,0.022041,-0.045333,0.055118,-0.200431,0.003236,0.019737,...,-0.024390,-0.038462,0.003236,0.019737,0.000000,0.032258,0.000000,0.090909,0.024996,-0.008784
2015-01-08,0.039543,0.036997,0.056604,0.000000,0.033175,-0.016447,0.076696,-0.020333,-0.004938,-0.050566,...,0.021277,0.000000,-0.004938,-0.050566,0.000000,0.032258,0.111801,0.098592,0.038551,0.018716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-25,0.007344,-0.046053,0.046512,0.037137,-0.006508,0.010944,0.018868,0.004082,-0.007786,-0.010914,...,0.020000,0.044643,-0.007786,-0.010914,-0.097561,0.105263,-0.024390,0.000000,0.008264,0.008256
2024-09-26,-0.025063,-0.051630,0.035892,0.052690,-0.043418,-0.118343,-0.018692,-0.029289,-0.049317,-0.071913,...,0.099010,0.055319,-0.049317,-0.071913,-0.012987,0.256410,-0.053571,-0.040000,-0.007133,-0.014985
2024-09-27,-0.003115,-0.039604,0.027545,0.016828,-0.014514,-0.033708,-0.058480,0.000000,-0.003713,-0.050633,...,0.109589,0.041322,-0.003713,-0.050633,-0.071429,0.125000,-0.051724,0.047619,-0.002126,-0.000927


In [46]:
# Weighted Sentiment Score for S&P 500

weights = {
    "AAPL": sp500_companies.loc[0, 'Market Cap Percentage'],
    "NVDA": sp500_companies.loc[1, 'Market Cap Percentage'],
    "MSFT": sp500_companies.loc[2, 'Market Cap Percentage'],
    "AMZN": sp500_companies.loc[3, 'Market Cap Percentage'],
    "GOOGL": sp500_companies.loc[4, 'Market Cap Percentage'],
    "META": sp500_companies.loc[5, 'Market Cap Percentage'],
    "TSLA": sp500_companies.loc[6, 'Market Cap Percentage'],
    "AVGO": sp500_companies.loc[7, 'Market Cap Percentage'],
    "COST": sp500_companies.loc[8, 'Market Cap Percentage'],
    "NFLX": sp500_companies.loc[9, 'Market Cap Percentage'],
    "TMUS": sp500_companies.loc[10, 'Market Cap Percentage'],
    "ASML": sp500_companies.loc[11, 'Market Cap Percentage'],
    "CSCO": sp500_companies.loc[12, 'Market Cap Percentage'],
    "ADBE": sp500_companies.loc[13, 'Market Cap Percentage'],
    "PEP": sp500_companies.loc[14, 'Market Cap Percentage']
}

# Function to calculate weighted sentiment score
def calculate_weighted_sentiment(sp, weights, sentiment_type):
    weighted_sentiment = []
    
    # Calculate weighted sentiment score for each date
    for index, row in sp.iterrows():
        weighted_score = 0
        total_weight = 0
        for stock, weight in weights.items():
            sentiment_column = f'{stock} {sentiment_type} Sentiment Score'
            if sentiment_column in sp.columns:
                weighted_score += row[sentiment_column] * weight
                total_weight += weight
        # Store weighted sentiment for each date
        weighted_sentiment.append(weighted_score / total_weight if total_weight > 0 else 0)
    
    return weighted_sentiment

# Calculate the weighted sentiment scores for News and Twitter
sp['Weighted News Sentiment Score'] = calculate_weighted_sentiment(sp, weights, 'News')
sp['Weighted Twitter Sentiment Score'] = calculate_weighted_sentiment(sp, weights, 'Twitter')

# Display the results
sp

,AAPL News Sentiment Score,AAPL Twitter Sentiment Score,NVDA News Sentiment Score,NVDA Twitter Sentiment Score,MSFT News Sentiment Score,MSFT Twitter Sentiment Score,AMZN News Sentiment Score,AMZN Twitter Sentiment Score,GOOGL News Sentiment Score,GOOGL Twitter Sentiment Score,...,JPM News Sentiment Score,JPM Twitter Sentiment Score,V News Sentiment Score,V Twitter Sentiment Score,UNH News Sentiment Score,UNH Twitter Sentiment Score,XOM News Sentiment Score,XOM Twitter Sentiment Score,Weighted News Sentiment Score,Weighted Twitter Sentiment Score
Date,,,,,,,,,,,,,,,,,,,,,
2024-09-30,-0.005988,-0.043243,-0.011696,0.008111,-0.006303,-0.029536,-0.011976,-0.027273,-0.073840,-0.048780,...,0.000000,-0.018868,-0.105263,-0.147059,0.000000,-1.000000,-0.008264,-0.214286,-0.014649,-0.020402
2024-09-27,-0.003115,-0.039604,0.027545,0.016828,-0.014514,-0.033708,-0.058480,0.000000,-0.003713,-0.050633,...,-0.008242,0.112500,-0.063636,-0.140625,-0.017544,-0.090909,-0.097902,-0.175141,-0.002731,-0.012383
2024-09-26,-0.025063,-0.051630,0.035892,0.052690,-0.043418,-0.118343,-0.018692,-0.029289,-0.049317,-0.071913,...,-0.006110,0.038462,-0.327731,-0.105000,-0.115385,0.000000,-0.061947,-0.237569,-0.007512,-0.024673
2024-09-25,0.007344,-0.046053,0.046512,0.037137,-0.006508,0.010944,0.018868,0.004082,-0.007786,-0.010914,...,0.000000,-0.034043,-0.430913,-0.328358,0.142857,0.000000,-0.098131,-0.536585,0.015739,0.003237
2024-09-24,-0.001019,-0.025000,0.032206,0.012435,-0.021601,-0.045330,0.012232,0.047945,-0.015385,-0.026761,...,0.000000,0.021186,-0.291667,-0.364583,-0.028571,-0.200000,-0.286058,-0.706237,0.013249,-0.003862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-08,0.039543,0.036997,0.056604,0.000000,0.033175,-0.016447,0.076696,-0.020333,-0.004938,-0.050566,...,0.017391,0.036900,0.000000,0.000000,0.031250,-0.322581,0.026119,-0.046512,0.035358,0.009673
2015-01-07,0.036989,0.005421,0.006018,0.029394,0.022041,-0.045333,0.055118,-0.200431,0.003236,0.019737,...,0.012585,0.085774,0.071429,0.025316,0.000000,0.041667,-0.044983,-0.092391,0.029824,-0.020919
2015-01-06,0.018311,-0.068352,0.006018,0.009434,0.043785,0.010517,0.084906,0.030501,0.006747,-0.078540,...,-0.001471,-0.210454,-0.083333,0.000000,0.000000,-0.045455,0.010949,-0.233333,0.027856,-0.013235


In [ ]:
# Save the final Nasdaq 100 DataFrame to a CSV file
ndx.to_csv('ndx_weighted_sentiment_scores.csv', index=True)

# Save the final S&P 500 DataFrame to a CSV file
sp.to_csv('sp_weighted_sentiment_scores.csv', index=True)